# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [1]:
from xml.etree import ElementTree as ET

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [2]:
document_tree = ET.parse( './data/mondial_database_less.xml' )

In [5]:
# print names of all countries
for child in document_tree.getroot():
    print(child.find('name').text)

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra


In [6]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    print ('* ' + element.find('name').text + ':'),
    capitals_string = ''
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print (capitals_string[:-2])

* Albania:
Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
* Greece:
Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
* Macedonia:
Skopje, Kumanovo
* Serbia:
Beograd, Novi Sad, Niš
* Montenegro:
Podgorica
* Kosovo:
Prishtine
* Andorra:
Andorra la Vella


****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [44]:
import pandas as pd
import numpy as np
document = ET.parse( './data/mondial_database.xml' )

In [62]:
#10 countries with the lowest infant mortality rates

#intialise country and infant_mortality lists to populate
country=[]
infant_mortality=[]

#populate country and infant_mortality lists from XML
for element in document.iterfind('country'):
    country.append(element.find('name').text)
    #need exception handling for countries with no infant mortality data
    try:
        infant_mortality.append(element.find('infant_mortality').text)
    except:
        infant_mortality.append(np.nan)

#create pandas dataframe from country and infant_mortality lists
infant_mortality_df = pd.DataFrame({'country':country,'infant_mortality':infant_mortality})

#convert infant_mortality to numeric
infant_mortality_df.infant_mortality = infant_mortality_df.infant_mortality.convert_objects(convert_numeric=True)

In [96]:
len(country)

244

In [80]:
#show 10 countries with the lowest infant mortality rates
infant_mortality_df.sort('infant_mortality').head(10)

,country,infant_mortality
38,Monaco,1.81
98,Japan,2.13
117,Bermuda,2.48
36,Norway,2.48
106,Singapore,2.53
37,Sweden,2.60
10,Czech Republic,2.63
78,Hong Kong,2.73
79,Macao,3.13
44,Iceland,3.15


In [163]:
#10 cities with the largest population

#initialise cities and population lists
cities=[]
population=[]

for element in document.iterfind('country'):
    for subelement in element.getiterator('city'):
        cities.append(subelement.find('name').text)
        # each city has multiple population figures, iterating and overwriting the population will mean that the latest figure is stored and appended to the population list
        for subsub in subelement.findall('population'):
            x=subsub.text
        population.append(x)
 
                


In [164]:
#create pandas dataframe from cities and population lists
city_population_df = pd.DataFrame({'city':cities,'population':population})

#convert infant_mortality to numeric
city_population_df.population = city_population_df.population.convert_objects(convert_numeric=True)

In [165]:
#show 10 cities with the largest populations

city_population_df.sort('population', ascending=False).head(10)

,city,population
1341,Shanghai,22315474
771,Istanbul,13710512
1527,Mumbai,12442373
479,Moskva,11979529
1340,Beijing,11716620
2810,São Paulo,11152344
1342,Tianjin,11090314
1064,Guangzhou,11071424
1582,Delhi,11034555
1067,Shenzhen,10358381


In [264]:
#10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)

#initialise ethnicgroup and population lists

ethnicgroup=[]
population=[]
for element in document.iterfind('country'):
    #store the latest overall population number for each country
    for pop in element.findall('population'):
        x=float(pop.text)
    #store the ethnic group name and the product of the percentage with the latest population number in the respective list
    for eth in element.findall('ethnicgroup'):
        ethnicgroup.append(eth.text)
        population.append(x*float(eth.attrib['percentage'])/100)
        
ethnic_df=pd.DataFrame({'ethnicgroup':ethnicgroup,'population':population})
     

In [271]:
ethnic_df.groupby('ethnicgroup').sum().sort('population', ascending=False).head(10)

,population
ethnicgroup,
Han Chinese,1.245059e+09
Indo-Aryan,8.718156e+08
European,4.948722e+08
African,3.183251e+08
Dravidian,3.027137e+08
Mestizo,1.577344e+08
Bengali,1.467769e+08
Russian,1.318570e+08
Japanese,1.265342e+08


In [313]:
#name and country of a) longest river, b) largest lake and c) airport at highest elevation

#a) longest river
r_length=[]
r_name=[]
r_country=[]


for element in document.iterfind('river'):
    try:
        r_length.append(float(element.find('length').text))
    except:
        r_length.append(np.nan)
    r_name.append(element.find('name').text)
    r_country.append(element.attrib['country'])
    
river_df=pd.DataFrame({'r_name':r_name,'r_length':r_length,'r_country':r_country})

river_df.sort('r_length', ascending=False).head(1)


,r_country,r_length,r_name
174,CO BR PE,6448,Amazonas


In [319]:
#b) biggest lake
l_area=[]
l_name=[]
l_country=[]


for element in document.iterfind('lake'):
    try:
        l_area.append(float(element.find('area').text))
    except:
        l_area.append(np.nan)
    l_name.append(element.find('name').text)
    l_country.append(element.attrib['country'])
    
lake_df=pd.DataFrame({'l_name':l_name,'l_area':l_area,'l_country':l_country})

lake_df.sort('l_area', ascending=False).head(1)


,l_area,l_country,l_name
54,386400,R AZ KAZ IR TM,Caspian Sea


In [320]:
# c) airport at highest elevation

a_elev=[]
a_name=[]
a_country=[]


for element in document.iterfind('airport'):
    try:
        a_elev.append(float(element.find('elevation').text))
    except:
        a_elev.append(np.nan)
    a_name.append(element.find('name').text)
    a_country.append(element.attrib['country'])
    
airport_df=pd.DataFrame({'a_name':a_name,'a_elev':a_elev,'a_country':a_country})

airport_df.sort('a_elev', ascending=False).head(1)

,a_country,a_elev,a_name
80,BOL,4063,El Alto Intl


In [296]:
#map country code to country names

countryname=[]
car_code=[]

for element in document.iterfind('country'):
    countryname.append(element.find('name').text)
    car_code.append(element.attrib['car_code'])
    
countrycodemap=pd.DataFrame({'countryname':countryname,'code':car_code})